In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

In [3]:
example="My name is Neda and I work at HuggingFace in Lisbon"
inputs=tokenizer(example,return_tensors="pt")
outputs=model(**inputs)

In [4]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 17])
torch.Size([1, 17, 9])


In [5]:
#outputs.logits[0]

In [6]:
import torch
probablities=torch.nn.functional.softmax(outputs.logits,dim=-1)[0].tolist()

predictions = outputs.logits.argmax(dim=-1)[0].tolist()

In [7]:
#print(probablities)
print(predictions)

[0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 6, 6, 6, 6, 0, 8, 0]


In [8]:
model.config.id2label[0]


'O'

In [9]:
results=[]
tokens=inputs.tokens()

In [10]:
predictions


[0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 6, 6, 6, 6, 0, 8, 0]

In [11]:
len(predictions)

17

In [12]:
tokens

['[CLS]',
 'My',
 'name',
 'is',
 'Ned',
 '##a',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 '##F',
 '##ace',
 'in',
 'Lisbon',
 '[SEP]']

In [13]:
probablities

[[0.9995946288108826,
  1.1904368875548244e-05,
  2.2082498617237434e-05,
  1.2704006621788722e-05,
  3.333123822812922e-05,
  1.8673663362278603e-05,
  0.00011463042028481141,
  1.3042336831858847e-05,
  0.00017897906946018338],
 [0.999697208404541,
  8.016510037123226e-06,
  2.2806332708569244e-05,
  4.8654546844772995e-06,
  9.687086276244372e-05,
  1.1681508112815209e-05,
  9.455468534724787e-05,
  1.0597861546557397e-05,
  5.3463358199223876e-05],
 [0.9998413324356079,
  4.965365860698512e-06,
  1.7453236068831757e-05,
  3.0970370517025003e-06,
  3.901646414306015e-05,
  5.78072695134324e-06,
  3.784393993555568e-05,
  7.183261459431378e-06,
  4.3410145735833794e-05],
 [0.9999212026596069,
  3.1805802791495807e-06,
  7.846408152545337e-06,
  2.24450104724383e-06,
  1.5077702300914098e-05,
  4.748419996758457e-06,
  2.48843316512648e-05,
  4.913389147986891e-06,
  1.584161327627953e-05],
 [0.0008780207135714591,
  6.208219565451145e-05,
  0.00024273501185234636,
  3.229597132303752

In [14]:
for idx, pred in enumerate(predictions):
    print(idx,pred)

0 0
1 0
2 0
3 0
4 4
5 4
6 0
7 0
8 0
9 0
10 6
11 6
12 6
13 6
14 0
15 8
16 0


In [15]:
for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probablities[idx][pred], "word": tokens[idx]}
        )

In [16]:
print(results)

[{'entity': 'I-PER', 'score': 0.9970694184303284, 'word': 'Ned'}, {'entity': 'I-PER', 'score': 0.9978504180908203, 'word': '##a'}, {'entity': 'I-ORG', 'score': 0.9985812902450562, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9928873181343079, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9982545971870422, 'word': '##F'}, {'entity': 'I-ORG', 'score': 0.9967711567878723, 'word': '##ace'}, {'entity': 'I-LOC', 'score': 0.9963699579238892, 'word': 'Lisbon'}]


In [17]:
inputs_with_offset=tokenizer(example,return_tensors="pt",return_offsets_mapping=True)

In [18]:
inputs_with_offset["offset_mapping"]

tensor([[[ 0,  0],
         [ 0,  2],
         [ 3,  7],
         [ 8, 10],
         [11, 14],
         [14, 15],
         [16, 19],
         [20, 21],
         [22, 26],
         [27, 29],
         [30, 32],
         [32, 37],
         [37, 38],
         [38, 41],
         [42, 44],
         [45, 51],
         [ 0,  0]]])

In [19]:
example[11:14]

'Ned'

In [20]:
results=[]
inputs_with_offset=tokenizer(example,return_offsets_mapping=True)
tokens=inputs_with_offset.tokens()
offsets=inputs_with_offset["offset_mapping"]


In [21]:
for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start,end=offsets[idx]
        
        results.append(
            {"entity": label,
             "score": probablities[idx][pred],
             "word": tokens[idx],
            "start":start,
            "end":end}
        )

In [22]:
results

[{'entity': 'I-PER',
  'score': 0.9970694184303284,
  'word': 'Ned',
  'start': 11,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9978504180908203,
  'word': '##a',
  'start': 14,
  'end': 15},
 {'entity': 'I-ORG',
  'score': 0.9985812902450562,
  'word': 'Hu',
  'start': 30,
  'end': 32},
 {'entity': 'I-ORG',
  'score': 0.9928873181343079,
  'word': '##gging',
  'start': 32,
  'end': 37},
 {'entity': 'I-ORG',
  'score': 0.9982545971870422,
  'word': '##F',
  'start': 37,
  'end': 38},
 {'entity': 'I-ORG',
  'score': 0.9967711567878723,
  'word': '##ace',
  'start': 38,
  'end': 41},
 {'entity': 'I-LOC',
  'score': 0.9963699579238892,
  'word': 'Lisbon',
  'start': 45,
  'end': 51}]

In [23]:
example[30:41]

'HuggingFace'

In [ ]:
import numpy as np

results=[]
inputs_with_offset=tokenizer(example,return_offsets_mapping=True)
tokens=inputs_with_offset.tokens()
offsets=inputs_with_offset["offset_mapping"]

idx=0
while idx<len(predictions):
    pred=predictions[idx]
    lable=model.config.id2label[pred]
    if label != "O":
        #remove -B or _I
        lable=lable[2:]
        start,_=offsets[idx]
        
        all_acores=[]
        while (
        idx<len(predcitions) and model.config.id2label[predictions[idx]]==f"I-{lable}"
        ):
            
            all_scores.append(probablities[idx][pred])
            _,end=offsets[idx]
            idx+=1
            # teh score is the mean of all scores with the tokens in the same group
            np.mean(all_scores).item()
            word=exmapel[start:end]
            resutls.append(
            {
             "entity": label,
             "score": probablities[idx][pred],
             "word": tokens[idx],
             "start":start,
             "end":end}
            )
            
            idx+=1
            
        
        
        
        
